# Summarizing and Aggregating

En este capítulo aprenderás sobre:

- El contexto de GroupBy y sus métodos disponibles, y cómo usarlos para analizar tus datos.
- Cómo agrupar datos basados en valores temporales usando los métodos `df.group_by_dynamic()`, `df.rolling()` y `Expr.over()`.
- Optimizaciones que puedes aplicar para mejorar el rendimiento.

---


## Split, Apply, and Combine

El enfoque "split, apply, combine" es fundamental para el análisis de datos agrupados. En Polars, este patrón se implementa de manera eficiente mediante el método `df.group_by()`. 

- **Split:** Separa el DataFrame en grupos según los valores de una o más columnas.
- **Apply:** Aplica funciones de agregación (como `sum()`, `mean()`, `count()`, etc.) a cada grupo.
- **Combine:** Une los resultados en un nuevo DataFrame, facilitando el análisis y la visualización.

Este método permite resumir grandes volúmenes de datos y extraer información relevante de manera rápida y reproducible. A continuación, veremos ejemplos prácticos de cómo utilizar `group_by` en Polars para realizar estas operaciones.

---

In [1]:
import polars as pl

## GroupBy Context

In [5]:
fruit = pl.read_csv("data/fruit.csv")
fruit_grouped = fruit.group_by("is_round")
fruit_grouped

In [6]:
fruit_grouped.len()

is_round,len
bool,u32
true,4
false,6


## Métodos disponibles en el contexto GroupBy

| Método                      | Descripción                                                                                   |
|-----------------------------|----------------------------------------------------------------------------------------------|
| GroupBy.__iter__()          | Permite iterar sobre los grupos (sub-DataFrames) generados por la operación `group_by`.      |
| GroupBy.agg(…)              | Calcula agregaciones para cada grupo (sub-DataFrame) de una operación `group_by`.            |
| GroupBy.all()               | Agrega los grupos en una Serie.                                                              |
| GroupBy.count()             | Devuelve el número de filas en cada grupo.                                                   |
| GroupBy.first()             | Agrega los primeros valores de cada grupo.                                                   |
| GroupBy.head(…)             | Obtiene las primeras n filas de cada grupo.                                                  |
| GroupBy.last()              | Agrega los últimos valores de cada grupo.                                                    |
| GroupBy.len(…)              | Devuelve el número de filas en cada grupo.                                                   |
| GroupBy.map_groups(…)       | Aplica una función personalizada (UDF) sobre los grupos como sub-DataFrame.                  |
| GroupBy.max()               | Reduce los grupos al valor máximo.                                                           |
| GroupBy.mean()              | Reduce los grupos al valor medio.                                                            |
| GroupBy.median()            | Devuelve la mediana por grupo.                                                               |
| GroupBy.min()               | Reduce los grupos al valor mínimo.                                                           |
| GroupBy.n_unique()          | Cuenta los valores únicos por grupo.                                                         |
| GroupBy.quantile(…)         | Calcula el cuantil por grupo.                                                                |
| GroupBy.sum()               | Reduce los grupos a la suma.                                                                 |
| GroupBy.tail(…)             | Obtiene las últimas n filas de cada grupo.                                                   |

- El método `df.set_sorted()` informa a Polars que la columna `positie` ya está ordenada. Esto permite que Polars aplique optimizaciones rápidas (fast path) que solo son posibles si sabe que los datos están ordenados.
- Estas optimizaciones mejoran el rendimiento al aprovechar el conocimiento sobre el orden de los datos.
- Es fundamental asegurarse de que la columna realmente esté ordenada antes de usar este método, ya que un uso incorrecto puede provocar resultados erróneos debido a suposiciones falsas sobre los datos.

In [7]:
top2000 = pl.read_excel(
    "data/top2000-2023.xlsx", read_options={"skip_rows": 1}
).set_sorted("positie")

In [9]:
top2000.head()

positie,titel,artiest,jaar
i64,str,str,i64
1,"""Bohemian Rhapsody""","""Queen""",1975
2,"""Roller Coaster""","""Danny Vera""",2019
3,"""Hotel California""","""Eagles""",1977
4,"""Piano Man""","""Billy Joel""",1974
5,"""Fix You""","""Coldplay""",2005


In [ ]:
# Agrupa las canciones por año y concatena artista y título en una sola cadena por grupo.
(
    top2000.group_by("jaar")
    .agg(
        songs=pl.concat_str(             # Concatena strings
            pl.col("artiest"), pl.lit(" - "), pl.col("titel")
        ),
    )
    .sort("jaar", descending=True)
)

jaar,songs
i64,list[str]
2022,"[""Son Mieux - Multicolor"", ""Bankzitters - Je Blik Richting Mij"", … ""Måneskin - THE LONELIEST""]"
2021,"[""Goldband - Noodgeval"", ""Bankzitters - Stapelgek"", … ""Olivia Rodrigo - Drivers License""]"
2020,"[""DI-RECT - Soldier On"", ""Miss Montreal - Door De Wind"", … ""Dua Lipa ft. DaBaby - Levitating""]"
2019,"[""Danny Vera - Roller Coaster"", ""Floor Jansen & Henk Poort - Phantom Of The Opera"", … ""Tino Martin - Zij Weet Het""]"
2018,"[""Lady Gaga & Bradley Cooper - Shallow"", ""White Lies - Time To Give"", … ""Calvin Harris & Dua Lipa - One Kiss""]"
…,…
1960,"[""Etta James - At Last"", ""Shadows - Apache""]"
1959,"[""Jacques Brel - Ne Me Quitte Pas"", ""Elvis Presley - Hound Dog""]"
1958,"[""Chuck Berry - Johnny B. Goode"", ""Ella Fitzgerald & Louis Armstrong - Summertime""]"


In [ ]:
(
    top2000.group_by("jaar", maintain_order=True) # Agrupar por año y mantener orden
    .head(3) # Tomar los top 3 de canciones por año
    .sort("jaar", descending=True) # Ajustar por año
    .head(9) # Tomar solo los 3 años más recientes su top3
)

jaar,positie,titel,artiest
i64,i64,str,str
2022,179,"""Multicolor""","""Son Mieux"""
2022,370,"""Je Blik Richting Mij""","""Bankzitters"""
2022,395,"""L'enfer""","""Stromae"""
2021,55,"""Noodgeval""","""Goldband"""
2021,149,"""Stapelgek""","""Bankzitters"""
2021,210,"""Dat Heb Jij Gedaan""","""Meau"""
2020,19,"""Soldier On""","""DI-RECT"""
2020,38,"""Door De Wind""","""Miss Montreal"""
2020,77,"""Impossible (Orchestral Version…","""Nothing But Thieves"""


In [ ]:
(
    top2000.group_by("jaar", maintain_order=True)
    .tail(3)
    .sort("jaar", descending=True)
    .head(9)
 )

jaar,positie,titel,artiest
i64,i64,str,str
2022,1391,"""De Diepte""","""S10"""
2022,1688,"""Zeit""","""Rammstein"""
2022,1716,"""THE LONELIEST""","""Måneskin"""
2021,1865,"""Bon Gepakt""","""Donnie & Rene Froger"""
2021,1978,"""Hold On""","""Armin van Buuren ft. Davina Mi…"
2021,2000,"""Drivers License""","""Olivia Rodrigo"""
2020,1824,"""Smoorverliefd""","""Snelle"""
2020,1879,"""The Business""","""Tiësto"""
2020,1902,"""Levitating""","""Dua Lipa ft. DaBaby"""


In [12]:
(top2000.group_by("artiest").len().sort("len", descending=True).head(10))

artiest,len
str,u32
"""Queen""",34
"""The Beatles""",31
"""ABBA""",25
"""The Rolling Stones""",22
"""Bruce Springsteen""",22
"""Michael Jackson""",20
"""Fleetwood Mac""",20
"""Coldplay""",20
"""David Bowie""",18


In [18]:
sales = pl.read_csv("data/sales.csv")
sales.columns

['Date',
 'Day',
 'Month',
 'Year',
 'Customer_Age',
 'Age_Group',
 'Customer_Gender',
 'Country',
 'State',
 'Product_Category',
 'Sub_Category',
 'Product',
 'Order_Quantity',
 'Unit_Cost',
 'Unit_Price',
 'Profit',
 'Cost',
 'Revenue']

In [19]:
(
    sales.select("Product_Category", "Sub_Category", "Unit_Price")  
        .group_by("Product_Category", "Sub_Category")  
        .max()
        .sort("Unit_Price", descending=True)  
        .head(10)
)

Product_Category,Sub_Category,Unit_Price
str,str,i64
"""Bikes""","""Road Bikes""",3578
"""Bikes""","""Mountain Bikes""",3400
"""Clothing""","""Vests""",2384
"""Bikes""","""Touring Bikes""",2384
"""Accessories""","""Bike Stands""",159
"""Accessories""","""Bike Racks""",120
"""Clothing""","""Socks""",70
"""Clothing""","""Shorts""",70
"""Accessories""","""Hydration Packs""",55


In [ ]:
(
    sales.select("Country", "Profit")
    .group_by("Country")
    .sum()
    .sort(by="Profit", descending=True)
)

Country,Profit
str,i64
"""United States""",11073644
"""Australia""",6776030
"""United Kingdom""",4413853
"""Canada""",3717296
"""Germany""",3359995
"""France""",2880282


In [21]:
(
    sales.select("Sub_Category", "Product")
    .group_by("Sub_Category")
    .n_unique()
    .sort("Product", descending=True)
    .head(10)
)

Sub_Category,Product
str,u32
"""Road Bikes""",38
"""Mountain Bikes""",28
"""Touring Bikes""",22
"""Tires and Tubes""",11
"""Jerseys""",8
"""Vests""",4
"""Gloves""",4
"""Socks""",3
"""Helmets""",3


In [22]:
(
    sales.select("Age_Group", "Order_Quantity")
    .group_by("Age_Group")
    .mean()
    .sort("Order_Quantity", descending=True)
)

Age_Group,Order_Quantity
str,f64
"""Seniors (64+)""",13.530137
"""Youth (<25)""",12.124018
"""Adults (35-64)""",12.045303
"""Young Adults (25-34)""",11.560899


In [23]:
(
    sales.select("Age_Group", "Revenue")
    .group_by("Age_Group")
    .quantile(0.9)
    .sort("Revenue", descending=True)
)

Age_Group,Revenue
str,f64
"""Young Adults (25-34)""",2227.0
"""Adults (35-64)""",2217.0
"""Youth (<25)""",1997.0
"""Seniors (64+)""",943.0


## Advanced Methods

In [25]:
(
    sales.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.col("Profit"),
        pl.col("Revenue"),
    )
)

Country,Profit,Revenue
str,list[i64],list[i64]
"""United States""","[524, 407, … 542]","[929, 722, … 878]"
"""Germany""","[160, 53, … 746]","[295, 98, … 1250]"
"""United Kingdom""","[1053, 1053, … 112]","[1728, 1728, … 184]"
"""Canada""","[590, 590, … 630]","[950, 950, … 1014]"
"""Australia""","[1366, 1188, … 655]","[2401, 2088, … 1183]"
"""France""","[427, 427, … 655]","[787, 787, … 1207]"


In [26]:
(
    sales.group_by("Country").agg(
        pl.col("Profit").alias("All Profits Per Transactions"),
        pl.col("Revenue").name.prefix("All "),
        Cost=pl.col("Revenue") - pl.col("Profit")
    )
)

Country,All Profits Per Transactions,All Revenue,Cost
str,list[i64],list[i64],list[i64]
"""Canada""","[590, 590, … 630]","[950, 950, … 1014]","[360, 360, … 384]"
"""Australia""","[1366, 1188, … 655]","[2401, 2088, … 1183]","[1035, 900, … 528]"
"""United Kingdom""","[1053, 1053, … 112]","[1728, 1728, … 184]","[675, 675, … 72]"
"""France""","[427, 427, … 655]","[787, 787, … 1207]","[360, 360, … 552]"
"""United States""","[524, 407, … 542]","[929, 722, … 878]","[405, 315, … 336]"
"""Germany""","[160, 53, … 746]","[295, 98, … 1250]","[135, 45, … 504]"


### Apply multiple aggregations at once

In [27]:
(    
    sales.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.col("Profit").sum().name.prefix("Total "),
        pl.col("Profit").mean().alias("Average Profit per Transaction"),
        pl.col("Revenue").sum().name.prefix("Total "),
        pl.col("Revenue").mean().alias("Average Revenue per Transaction")
    )
)

Country,Total Profit,Average Profit per Transaction,Total Revenue,Average Revenue per Transaction
str,i64,f64,i64,f64
"""France""",2880282,261.891435,8432872,766.764139
"""Canada""",3717296,262.187615,7935738,559.721964
"""Australia""",6776030,283.089489,21302059,889.959016
"""United Kingdom""",4413853,324.071439,10646196,781.659031
"""Germany""",3359995,302.756803,8978596,809.028293
"""United States""",11073644,282.447687,27975547,713.552696


In [28]:
(
    sales.select("Country", "Profit", "Revenue")
    .group_by("Country")
    .agg(
        pl.all().sum().name.prefix("Total "),
        pl.all().mean().name.prefix("Average "),
    )
)

Country,Total Profit,Total Revenue,Average Profit,Average Revenue
str,i64,i64,f64,f64
"""Germany""",3359995,8978596,302.756803,809.028293
"""United Kingdom""",4413853,10646196,324.071439,781.659031
"""United States""",11073644,27975547,282.447687,713.552696
"""France""",2880282,8432872,261.891435,766.764139
"""Canada""",3717296,7935738,262.187615,559.721964
"""Australia""",6776030,21302059,283.089489,889.959016
